# **Fake and Real news Prediction with NLP**

With data retrieved from **[Kaggle](https://www.kaggle.com/datasets/clmentbisaillon/fake-and-real-news-dataset?resource=download&select=True.csv)**, this jupyter performs NLP with two prediction methodologies.

- Politics only
- Balanced dataset

Possible explanation features:
- Citing
- Number of punctuation marks !?
- Proportion of CAPS out of all text

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [26]:
# Import the datasets
true_news = pd.read_csv('data/True.csv')
fake_news = pd.read_csv('data/Fake.csv')

In [27]:
# Get basic informattion for the true news
print(true_news.info(), '\n') # Print the news structure/info
print(true_news['subject'].unique(), '\n') # Print the unique subject of true news
true_news.head() # Print the news head

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21417 entries, 0 to 21416
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    21417 non-null  object
 1   text     21417 non-null  object
 2   subject  21417 non-null  object
 3   date     21417 non-null  object
dtypes: object(4)
memory usage: 669.4+ KB
None 

['politicsNews' 'worldnews'] 



,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [28]:
# Get basic information for fake news
print(fake_news.info(), '\n') # Print the structure of the dataset
print(fake_news['subject'].unique(), '\n') # Print the unique subjects of the fake news
fake_news.head() # Print the head of the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23481 entries, 0 to 23480
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    23481 non-null  object
 1   text     23481 non-null  object
 2   subject  23481 non-null  object
 3   date     23481 non-null  object
dtypes: object(4)
memory usage: 733.9+ KB
None 

['News' 'politics' 'Government News' 'left-news' 'US_News' 'Middle-east'] 



,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


As we can see, there is a difference of `subject` between the two datasets. This may affect the results...

In [29]:
# Add a label for identifying whether is a fake or true news
true_news['is_fake'] = 0 # Dummy label for true news
fake_news['is_fake'] = 1 # Dummy label for fake news

# Concatenate the datasets
news = pd.concat([true_news, fake_news]).reset_index().drop(columns='index')

news

,title,text,subject,date,is_fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",1
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",1
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",1
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",1


For the purpose of this analysis, only subjects explicitly labeled as `politics` will be take in account. Thus because 1) it was one easy-identifiable subject found both in the *true* and *false* dataset and 2) is thought that considering more categories that are not in both datasets, may bias the results.

In [30]:
politics_subject = ['politicsNews', 'politics'] # Create a list with the politics subjects
pnews = news.query('subject == @politics_subject') # Filter the news dataset by ONLY the 'politics' related subject
print(pnews['subject'].unique(), '\n') # Print the unique values, they MUST be 'politics'-related

# Print the size of the All-news dataset and the politics-only
print(f'All-News records: {news.shape[0]:,}')
print(f'Politics News records: {pnews.shape[0]:,}')
print(f'Politics News records as % All-News records: {pnews.shape[0] / news.shape[0]:.2%}\n')

print(pnews['is_fake'].value_counts()) # Get the distribution of fake and true news

pnews

['politicsNews' 'politics'] 

All-News records: 44,898
Politics News records: 18,113
Politics News records as % All-News records: 40.34%

0    11272
1     6841
Name: is_fake, dtype: int64


,title,text,subject,date,is_fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
37303,APPLE’S CEO SAYS RELIGIOUS FREEDOM LAWS ARE ‘D...,The gay mafia has a new corporate Don. This i...,politics,"Mar 31, 2015",1
37304,WATCH DIRTY HARRY REID ON HIS LIE ABOUT ROMNEY...,"In case you missed it Sen. Harry Reid (R-NV), ...",politics,"Mar 31, 2015",1
37305,OH NO! GUESS WHO FUNDED THE SHRINE TO TED KENNEDY,Nothing like political cronyism to make your s...,politics,"Mar 31, 2015",1
37306,BENGHAZI PANEL CALLS HILLARY TO TESTIFY UNDER ...,Does anyone really think Hillary Clinton will ...,politics,"Mar 31, 2015",1


As seen, the politics news represent 40% of the total news. Also, it is visible that the current politics dataset is unbalanced, being the *true* news almost the double of *fake* ones. For simplification purposes, the dataset for the final analysis will be balanced.

We will also define a train and test set for model evaluation.

In [31]:
# 5472 records represent ~ 80% of fake news
pnews_tt = pnews.query('is_fake == 0').sample(5472, random_state=22) # True train news
pnews_ft = pnews.query('is_fake == 1').sample(5472, random_state=22) # Fake train news
pnews_train = pd.concat([pnews_tt, pnews_ft]) # Concat the train set

print(pnews_train.shape, '\n') # Print the shape of the balanced-train dataset
print(pnews_train['is_fake'].value_counts())    # Print the distribution of the `is_fake` variable

pnews

(10944, 5) 

0    5472
1    5472
Name: is_fake, dtype: int64


,title,text,subject,date,is_fake
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0
...,...,...,...,...,...
37303,APPLE’S CEO SAYS RELIGIOUS FREEDOM LAWS ARE ‘D...,The gay mafia has a new corporate Don. This i...,politics,"Mar 31, 2015",1
37304,WATCH DIRTY HARRY REID ON HIS LIE ABOUT ROMNEY...,"In case you missed it Sen. Harry Reid (R-NV), ...",politics,"Mar 31, 2015",1
37305,OH NO! GUESS WHO FUNDED THE SHRINE TO TED KENNEDY,Nothing like political cronyism to make your s...,politics,"Mar 31, 2015",1
37306,BENGHAZI PANEL CALLS HILLARY TO TESTIFY UNDER ...,Does anyone really think Hillary Clinton will ...,politics,"Mar 31, 2015",1


In [32]:
# Define a test set

# Get the fake news sample
fake_test_ix = np.setdiff1d(pnews.query('is_fake == 1').index, pnews_ft.index) # Get the indexes difference (find those that are not within the train set)
pnews_ftest = pnews.loc[fake_test_ix]


# Get the true news sample
true_test_ix = np.setdiff1d(pnews.query('is_fake == 0').index, pnews_tt.index) # Get the indexes difference (find those that are not within the train set)
pnews_ttest = pnews.loc[true_test_ix].sample(fake_test_ix.size, random_state=22)

# Concat the test samples into the test set
pnews_test = pd.concat([pnews_ftest, pnews_ttest])

print(pnews_test.shape, '\n') # Print the size of the test set
print(pnews_test['is_fake'].value_counts(), '\n') # Print the distribution of fake vs. true news

pnews_test.head()

(2738, 5) 

1    1369
0    1369
Name: is_fake, dtype: int64 



,title,text,subject,date,is_fake
30473,GOTCHA! A Stammering John Podesta Stunned by a...,John Podesta is Hillary Clinton s former campa...,politics,18-Feb-18,1
30476,BUSTED! Maxine Waters’ Ties to Black Nationali...,We reported that Maxine Waters and members of ...,politics,18-Feb-18,1
30482,“THE TRUMP CURSE IS REAL”…Social Media Respond...,Two months before the Winter Olympics in Seoul...,politics,17-Feb-18,1
30488,TUCKER CARLSON Rips The Left On Gun Control Hy...,Tucker Carlson weighed in on the hysteria over...,politics,16-Feb-18,1
30493,TWO HIGH SCHOOL JROTC Members Tell INCREDIBLE ...,ABC News When two Marjory Stoneman Douglas Hi...,politics,15-Feb-18,1


<a name='1'></a>
## 1 - Process the Data

For any machine learning project, once you've gathered the data, the first step is to process it to make useful inputs to your model.
- **Remove noise / stopwords**: You will first want to remove noise from your data -- that is, remove words that don't tell you much about the content. These include all common words like 'I, you, are, is, etc...' that would not give us enough information on the sentiment.
- We'll also remove stock market tickers, retweet symbols, hyperlinks, and hashtags because they can not tell you a lot of information on the sentiment.
- You also want to remove all the punctuation from a tweet. The reason for doing this is because we want to treat words with or without the punctuation as the same word, instead of treating "happy", "happy?", "happy!", "happy," and "happy." as different words.
- Finally you want to use stemming to only keep track of one variation of each word. In other words, we'll treat "motivation", "motivated", and "motivate" similarly by grouping them within the same stem of "motiv-".

We have given you the function `process_tweet` that does this for you.

In [33]:
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.tokenize import word_tokenize
import string
import re

# Found issues

- Abbreviations such as **CBS** will be lost during the string process.
- ASCII emojies like `:)`, `:(` will be lost udirng the string process.

In [45]:
def process_string(s, nomenclature=True):
    """
    Process a string by eliminating stopwords, tokenizing and stemming the words.

    # Arguments
    s: str
        A string to be processed.

    # Return
    clean_tokens: list
        A list with clean tokens.
    """

    clean_tokens = [] # Set an empty list for cleaned tokens

    # Validate if we want to validate first the existence of nomenclature
    if nomenclature:
        nom_regex = '(?:[A-Z]\.){1,}'               # A regex for identifying nomenclature like 'U.S.'
        nom_regex_find = re.findall(nom_regex, s)   # Find all the occurences for the nomeclature regex
        if len(nom_regex_find) > 0:                 # Validate that there is at least one result within the find
            clean_tokens += nom_regex_find          # Append the found nomenclatures into the clean_tokens list
            s = re.sub(nom_regex, '', s)            # Replace the found nomeclanture with empty spaces, wont be considered for tokenizing

    non_word_regex = '[^\w]'                                # Create a regex for identifying non-words, e.g. '!', '’', '.' ...
    s_words_only = re.sub(non_word_regex, ' ', s.lower())   # Replace non-word symbols with an empty space

    stemmer = PorterStemmer()                       # Init a Porter stemmer, gets the root/stem of the words
    en_stopwords = stopwords.words('english')       # Store the english stopwords, they are not as relevant in a sentiment 
    tokens = word_tokenize(s_words_only)            # Create a list with tokens, using the nltk.word_tokenize

    # Create a list of clean tokens by stemming each word
    # if the word is not within the unwanted words. (e.g. stopwords/punctuation)
    clean_tokens += [
        stemmer.stem(word) for word in tokens
        if word not in en_stopwords
        if not re.match(non_word_regex, word)
    ]

    return clean_tokens

In [48]:
def build_freqs(strings, sentiments):
    """
    Build frequencies of tokens/words given its sentiment.
    
    strings: list
        A list of strings/sentences.
    sentiments: list
        A list with the sentiment of the strings/sentences.
    """

    # Start with an empty dictionary and populate it with the the pairs of combinations (word, sentiment).
    freqs = {}

    # Iterate over each string and its sentiment
    for s, sentiment in zip(strings, sentiments):
        clean_tokens = process_string(s) # Process the string and get its clean tokens
        for word in clean_tokens:
            pair = (word, sentiment)    # Create a a tuple with the `item` and its `sentiment`
            if pair in freqs.keys():    # Identify whether or not the pair is within the keys
                freqs[pair] += 1
            else:
                freqs[pair] = 1

    return freqs

In [58]:
# Get the freqs from the political news dataset
freqs = build_freqs(pnews_test.title, pnews_test.is_fake)
# Get the TOP 10 words with more frequency
list({k:v for k,v in sorted(freqs.items(), key=lambda item: item[1], reverse=True)}.items())[:10]

[(('video', 1), 697),
 (('trump', 0), 626),
 (('trump', 1), 491),
 (('U.S.', 0), 380),
 (('say', 0), 208),
 (('hillari', 1), 197),
 (('obama', 1), 186),
 (('hous', 0), 177),
 (('senat', 0), 150),
 (('republican', 0), 120)]

In [59]:
# Get the Vocabulary of the dataset (set of unique words)
V = list(set([pair[0] for pair in freqs.keys()]))
print(f"Vocabulary's length: {len(V):,}") # Print the Vocabulary's length, count of unique words within our news title dataset
V[:5] # Print the head of the Vocabulary

Vocabulary's length: 4,719


['extremist', '2000', 'coron', 'sunday', 'chines']

In [60]:
# Get the top 50 words for each of the classes (fake and NON-fake ones)
fake_50 = {pair:count for pair, count in sorted(freqs.items(), key=lambda item: item[1], reverse=True) if pair[1] == 1} # Get only the items that have a is_fake == 1
list(fake_50.items())[:5] # Get the head of the fake_50

[(('video', 1), 697),
 (('trump', 1), 491),
 (('hillari', 1), 197),
 (('obama', 1), 186),
 (('clinton', 1), 107)]

In [61]:
# Get the top 50 words for non-fake title news
non_fake50 = {pair:count for pair, count in sorted(freqs.items(), key=lambda item: item[1], reverse=True) if pair[1] == 0}
list(non_fake50.items())[:5] # Get the head of non-fake

[(('trump', 0), 626),
 (('U.S.', 0), 380),
 (('say', 0), 208),
 (('hous', 0), 177),
 (('senat', 0), 150)]

# NEXT STEPS
- Visualize in by word frequency and sentiment

## **Prediction with Logistic Regression**

In this section, we will forecast whether the news is *true* or *fake* with *logistic regression*.